# Knapsack problem
## Definición del Problema: La Mochila (Knapsack Problem)

El **Problema de la Mochila** (*Knapsack Problem*) es un problema fundamental en optimización combinatoria y teoría de la complejidad, clasificado como NP-Hard. Debido a la explosión combinatoria de su espacio de búsqueda ($2^n$), es un candidato ideal para ser resuelto mediante Quantum Annealing en procesadores D-Wave.

### Formulación Clásica

Dado un conjunto de $n$ elementos, donde cada elemento $i$ posee un valor $v_i$ y un peso $w_i$, y una capacidad máxima de peso $W$, el objetivo es seleccionar un subconjunto de elementos tal que se maximice el valor total sin exceder la capacidad.

Definimos un vector binario de decisión $x$, donde $x_i \in \{0, 1\}$.

**Función Objetivo (Maximizar):**
$$\max \sum_{i=1}^{n} v_i x_i$$

**Sujeto a la restricción:**
$$\sum_{i=1}^{n} w_i x_i \le W$$

En definitiva, estamos buscando optimizar el llenado de una mochila eligiendo objetos de un conjunto. Cada objeto tiene un peso y un valor, y queremos maximizar el valor introducido en la mochila sin superar el peso máximo que la mochila puede soportar.


## I. La solución rápida: todo resuelto por las bibliotecas dimod

El problema de la mochila está implementado de forma nativa en la biblioteca de generadores de dimod para D-Wave. A continuación se muestra una implementación del problema con código mínimo, en el que se genera de forma aleatoria una configuración para el problema de la mochila y se resuelve. El ExactCQMSolver resuelve todas las posibles configuraciones, y luego podemos filtrar para elegir la mejor solución (menor energía) que satisface las restricciones (peso máximo).

In [ ]:
from dimod.generators import random_knapsack
from dimod import ExactCQMSolver
import pandas as pd
import numpy as np

value_range = (10,30) #Los objetos tendrán un valor entre 10 y 30
weight_range = (10,30) #Los objetos tendrán un peso (coste) entre 10 y 30
tightness_ratio = 0.5 # La mochila no puede llevar todo el peso, aproximadamente el doble de peso en los objetos del que podemos meter en la mochila
num_objects = 10 # Habrá un total de 10 objetos entre los que elegir
cqm = random_knapsack(num_objects, value_range=value_range, weight_range=weight_range, tightness_ratio=tightness_ratio)

sampler = ExactCQMSolver()
sampleset = sampler.sample_cqm(cqm)
df = sampleset.to_pandas_dataframe() #Cada fila del Dataframe es una solución

df_filtrado = df[(df['is_feasible'] == True) & (df['is_satisfied'] == True)] #filtramos las soluciones y nos quedamos solo con las que nos interesan
mejor_solucion = df_filtrado.loc[df_filtrado['energy'].idxmin()] #Tras filtrar, nos quedamos con la mejor (la que menor energía tenga)
print("Solución")
print(mejor_solucion)

## II. Desarrollando la solución
El objetivo de esta asignatura es desarrollar nosotros la solución al problema planteado. Para ello, debemos plantear el problema como un modelo Ising o QUBO.

Por definición, el knapsack problem es un problema de tipo "Binary CQM", es decir, Constrained Quadratic Model de tipo binario. Las variables son binarias (los objetos se eligen o no para estar en la mochila) y además de la formulación de la energía (el valor que se quiere minimizar), se debe introducir una restricción adicional para evitar que se supere el peso máximo de la mochila. Sin esa restricción, la solución siempre sería elegir todos los objetos.

Veamos como se generaría la formulación del problema, partiendo de los objetos, sus pesos y sus valores.

### Una clase para encapsular los problemas knapsack

Vamos a construir una clase para encapsular la definición de un problema de la mochila y su posible solución, y facilitar la legibilidad del resto del código.


In [ ]:

class KnapsackItem:
    """Clase que representa un objeto individual dentro de la mochila"""
    def __init__(self, name, weight, value, is_slack=False):
        self.name = name
        self._weight = weight
        self._value = value
        self.is_slack = is_slack # Propiedad para identificar si es holgura

    def weight(self):
        return self._weight

    def value(self):
        return self._value

    def __str__(self):
        return self.name

class ProblemaKnapsack:
    def __init__(self, max_weight=0):
        self.objects = [] # Usaremos objetos KnapsackItem
        self.max_weight = max_weight

    def add_object(self, name, weight, value):
        new_item = KnapsackItem(name, weight, value, is_slack=False)
        self.objects.append(new_item)

    def add_slack_object(self, name, weight):
        """Añade un objeto de holgura (sin valor, marcado como slack)"""
        new_item = KnapsackItem(name, weight, 0, is_slack=True)
        self.objects.append(new_item)

    def print_problem(self, mostrar_holgura=False):
        print("Definición del problema")

        # Filtramos objetos para mostrar usando el atributo is_slack del objeto
        objs_to_show = [o for o in self.objects if mostrar_holgura or not o.is_slack]
        print(tabulate( ([o.name, o.weight(), o.value()] for o in objs_to_show),
                        headers=["Objeto","Peso","Valor"]))
        print("Peso máximo: ", self.max_weight)

    def print_solution(self, solution, mostrar_holgura=False):
        print("Solución")

        # Helper para obtener valor de la solución
        def get_selection_status(item_name):
            if hasattr(solution, 'get'):
                return int(solution.get(item_name, 0))
            return 0 # Si falla por índice o tipo, asumimos 0

        # Preparamos datos tabla
        table_data = []
        total_weight = 0
        total_value = 0
        slack_weight_used = 0

        for o in self.objects:
            selected = get_selection_status(o.name)
            if not o.is_slack:
                total_weight += o.weight() * selected
                total_value += o.value() * selected
            else:
              slack_weight_used += o.weight()*selected

            if mostrar_holgura or not o.is_slack:
                table_data.append([o.name, o.weight(), o.value(), selected])

        print(tabulate(table_data, headers=["Objeto","Peso","Valor", "Seleccionado"]))
        print("Peso máximo permitido: ", self.max_weight)
        print("Peso total ocupado: ", total_weight)
        if mostrar_holgura:
            print("Holguras: ", slack_weight_used," -- Peso total utilizado incluyendo holgura: ", slack_weight_used+total_weight)
        print("Valor total conseguido: ", total_value)


### Generando un problema aleatorio
En primer lugar, generemos un problema aleatorio de modo que definamos directamente la lista de objetos y sus características (peso y valor), así como el peso máximo.

Para facilitar la gestión de estos problemas y sus soluciones, vamos a empaquetarlo todo en una clase

In [ ]:
import random
from tabulate import tabulate
import math

def random_knapsack_definition(num_objects, value_range, weight_range, tightness_ratio):
    problema = ProblemaKnapsack()

    total_weight_generated = 0

    for i in range(0, num_objects):
        name = "x_" + str(i)
        w = random.randint(weight_range[0], weight_range[1])
        v = random.randint(value_range[0], value_range[1])

        problema.add_object(name, w, v)
        total_weight_generated += w

    problema.max_weight = math.ceil(tightness_ratio * total_weight_generated)

    return problema

# Generamos el problema
problema = random_knapsack_definition(num_objects, value_range, weight_range, tightness_ratio)
problema.print_problem()

### Formulando el problema como un CQM
Debemos formular este problema como
- Una función cuadrática que determina el valor a **minimizar** (el valor de todos los objetos en la mochila, para una determinada solución)
- Una restricción que impide que el peso total supere el máximo permitido

Estas son las que hemos visto anteriormente en la formulación clásica del problema:

**Función Objetivo (minimizar):**
$$ - \sum_{i=1}^{n} v_i x_i$$

**Restricción:**
$$\sum_{i=1}^{n} w_i x_i \le W$$



In [ ]:
from dimod import Binary, ConstrainedQuadraticModel

valor_total = 0
peso_total = 0

for o in problema.objects:
    # Creamos la variable binaria usando el nombre del objeto
    v = Binary(o.name)

    valor_total += v * o.value()
    peso_total += v * o.weight()

cqm = ConstrainedQuadraticModel()
cqm.set_objective(-valor_total)
cqm.add_constraint(peso_total <= problema.max_weight)

sampler = ExactCQMSolver()
sampleset = sampler.sample_cqm(cqm)
df = sampleset.to_pandas_dataframe()

df_filtrado = df[(df['is_feasible'] == True) & (df['is_satisfied'] == True)]
mejor_solucion = df_filtrado.loc[df_filtrado['energy'].idxmin()]
print("Solución CQM Manual")
print(mejor_solucion)


### Formulando el problema como un BQM
No siempre podemos recurrir a los CQM e introducir restricciones, ya que cuando usamos ordenadores cuánticos para resolver, puede que estos implementen de forma nativa el modelo Ising, que no permite incorporar restricciones.

Podemos formular este problema directamente como un Binary Quadratic Model, un modelo sin restricciones, también binario. Para ello incorporamos la restricción como una penalización en la función de energía.

Esta restricción se puede implementar como el cuadrado de la diferencia entre el peso total y el peso máximo, usando un parámetro, $\alpha$, como factor de ponderación. El cuadrado nos permite que la restricción no tenga nunca un peso negativo. De este modo, se obligará al modelo a intentar aproximarse a la carga máxima (en ese caso la restricción vale cero).  

**Función Objetivo:**
$$ \underbrace{-\sum_{i=1}^{n} v_i x_i}_{\text{Función objetivo anterior}} + \underbrace{\alpha \left( \sum_{i=1}^{n} w_i x_i - W \right)^2}_{\text{Penalización}}$$

Se puede desarrollar el cuadrado, obteniendo:

$$ -\sum_{i=1}^{n} v_i x_i + \alpha \left( \sum_{i=1}^{n}\sum_{j=1}^{n} w_i w_j x_i x_j  + W^2 -2 W \sum_{i=1}^{n} w_i x_i \right)$$

Cabe destacar que en el doble sumatorio se puede dar el caso de $i=j$. Dicho término no sería cuadrático sino lineal, ya que, al ser $x_i$ binario, $x_i^2 = x_i$. Obtendríamos por tanto

$$ -\sum_{i=1}^{n} v_i x_i + \alpha \left( \sum_{i=1}^{n} w_i^2 x_i^2 + \sum_{i=1}^{n}\sum_{\substack{j=1 \\ j \neq i} }^{n} w_i w_j x_i x_j -2 W \sum_{i=1}^{n} w_i x_i + W^2 \right)$$

Que, simplificando, equivaldría a:

$$ -\sum_{i=1}^{n} v_i x_i + \alpha \left( \sum_{i=1}^{n} w_i^2 x_i + \sum_{i=1}^{n}\sum_{\substack{j=1 \\ j \neq i} }^{n} w_i w_j x_i x_j -2 W \sum_{i=1}^{n} w_i x_i \right)$$

Esta formulación se puede reorganizar para encajar en la formulación estándar de un QUBO, con todos los términos lineales agrupados y los cuadráticos agrupados:

$$ \sum_{i=0}^{n}(-v_i + \alpha w_i^2 -2 \alpha W w_i)x_i + \sum_{i=1}^{n}\sum_{\substack{j=1 \\ j \neq i} }^{n} \alpha w_i w_j x_i x_j$$

#### Límites de esta solución
La implementación de la restricción como un cuadrado de la distancia al peso máximo impone ciertas limitaciones. Este método se suele utilizar para implementar restricciones de igualdad, ya que con un parámetro $\alpha$ suficientemente grande, se forzará la búsqueda de una solución con el peso exacto.

Si bien este método es sencillo de implementar, no es óptimo para restricciones de desigualdad.
- Se pueden aceptar soluciones suboptimas a cambio de llenar la mochila
- En caso de no forzar la igualdad (parámetro $\alpha$ pequeño), se pueden obtener soluciones por encima del peso máximo.

Veamos, en cualquier caso, la aproximación simplificada BQM con restricciones débiles.

In [ ]:
from dimod import ExactSolver

qubo_basico = {}
alpha = 1

for o_i in problema.objects:
    # Términos lineales (la diagonal de la matriz QUBO)

    qubo_basico[(o_i.name, o_i.name)] = (
        -o_i.value()
        + alpha * (o_i.weight()**2)
        - 2 * alpha * problema.max_weight * o_i.weight()
    )

  # Términos cuadráticos (los elementos fuera de la diagonal de la matriz QUBO)
    for o_j in problema.objects:
        if o_i.name == o_j.name:
          continue
          # Aplicamos la penalización
        qubo_basico[(o_i.name, o_j.name)] = alpha * o_i.weight() * o_j.weight()

exact_solver = ExactSolver()
solucion_qubo_basico = exact_solver.sample_qubo(qubo_basico).first.sample

problema.print_solution(solucion_qubo_basico)



#### Calculando $\alpha$
El parámetro de ponderación (alpha en nuestro código) es clave para obtener soluciones óptimas. Un $\alpha$ demasiado pequeño permitirá incumplir la restricción, mientras que un $\alpha$ demasiado grande hará irrelevante el valor a maximizar (o coste a minimizar).

Debe elegirse un $\alpha$ tal que la penalización por violar la restricción siempre sea mayor que la ganancia obtenida por violarla. Puesto que la ganancia se determina en nuestro caso por el valor de los objetos, un posible $\alpha$ sería la suma de todos los valores de todos los objetos. No obstante, para problemas de gran tamaño podemos obtener un $\alpha$ demasiado grande.

En general, una buena heurística es tomar un $\alpha$ ligeramente superior al valor del mayor objeto.

El código en ese caso sería el siguiente:



In [ ]:
qubo_basico = {}

# Calculamos alpha buscando el valor máximo entre los objetos
alpha = max(o.value() for o in problema.objects) * 1.1

for o_i in problema.objects:
    # Término lineal
    qubo_basico[(o_i.name, o_i.name)] = (
        - o_i.value()
        + alpha * (o_i.weight()**2)
        - 2 * alpha * problema.max_weight * o_i.weight()
    )

    for o_j in problema.objects:
        if o_i.name == o_j.name:
          continue
        # Término cuadrático
        qubo_basico[(o_i.name, o_j.name)] = alpha * o_i.weight() * o_j.weight()

solucion_qubo_basico = exact_solver.sample_qubo(qubo_basico).first.sample

problema.print_solution(solucion_qubo_basico)




#### BQM Avanzado: restricciones fuertes, soluciones óptimas
Para resolver los problemas propios de la formulación simplificada del BQM, se utiliza la técnica conocida como "Slack Variables" o variables de holgura.

La idea es incluir "objetos" que tienen peso pero no añaden valor y nos permiten llenar el espacio vacío de la mochila. De este modo, para cualquier solución que llene la mochila de forma parcial y tenga un máximo de valor, existirá una solución de igual valor con objetos fantasma (sin valor), que ocupará exactamente la mochila completa. El peso total sería:

$$\sum_{i=1}^{n} w_i x_i + S = W$$

donde $S$ es exactamente el espacio vacío en esta solución.

Como los modelos QUBO solo entienden variables binarias (0 o 1), no podemos introducir $S$ directamente como un entero. Debemos descomponer $S$ en una suma de variables binarias auxiliares, $y_k$, que representan un conjunto de objetos que se pueden usar para llenar la mochila. Para asegurar que tenemos suficientes objetos para llenar la mochila justo hasta su límite, usamos potencias de $2$ desde $0$ hasta $M = \log_2 W$), para poder representar cualquier peso añadido hasta el peso máximo.

$$S = \sum_{k=0}^{M} 2^k y_k$$

Por tanto, la función de energía busca minimizar:

$$\underbrace{-\sum_{i=1}^{n} v_i x_i}_{\text{Valor}} + \alpha \underbrace{\left( \sum_{i=1}^{n} w_i x_i + \sum_{k=0}^{M} 2^k y_k - W \right)^2}_{\text{Restricción (Peso + Holgura - Peso máximo)}}$$

Si bien podríamos expandir el cuadrado anterior, la expresión adquiere cierta complejidad. Optaremos alternativamente por implementarlo añadiendo $M$ objetos, $y_k$, que tendrán peso $2^k$ y valor $0$.

Este método nos garantiza una solución a las dos limitaciones indicadas anteriormente, implementando efectivamente una restricción de desigualdad:

- Permite no llenar la mochila: Si la solución óptima pesa 5kg y la capacidad es 10kg, las variables $y_k$ tomarán el valor 5, haciendo que el término de penalización sea $(5+5-10)^2 = 0$. La energía es mínima y válida.

- Impide exceder el peso (Restricción Fuerte): No existen objetos con pesos negativos. Asumiendo que el valor $\alpha$ es suficientemente grande, cualquier solución que exceda el peso tendrá una penalización mayor que la ventaja de sobrepasar el peso.



In [ ]:
import copy

problema_holgura = copy.deepcopy(problema)

# Añadimos variables de holgura
smallest_object = min([o.weight() for o in problema_holgura.objects])
# Necesitamos holgura para compensar el espacio libre máximo, asumiendo que al menos el objeto más ligero
# está incluído. Tenemos que sumar 1 ya que si el peso máximo es justo una potencia de dos
# no cubriríamos todo el espacio (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
k = int(math.floor(math.log2(problema_holgura.max_weight - smallest_object)+1))
for k in range(0, k):
    o_name = "y_" + str(k)
    problema_holgura.add_slack_object(o_name, 2**k)

qubo_holgura = {}
alpha = max(o.value() for o in problema_holgura.objects) * 1.1

for o_i in problema_holgura.objects:
    qubo_holgura[(o_i.name, o_i.name)] = (
        - o_i.value()
        + alpha * (o_i.weight()**2)
        - 2 * alpha * problema_holgura.max_weight * o_i.weight()
    )

    for o_j in problema_holgura.objects:
        if o_i.name == o_j.name:
          continue
        qubo_holgura[(o_i.name, o_j.name)] = alpha * o_i.weight() * o_j.weight()

solucion_qubo_holgura = exact_solver.sample_qubo(qubo_holgura).first.sample

print("---- Solución normal (Slack oculta) ----")
problema_holgura.print_solution(solucion_qubo_holgura, mostrar_holgura=False)

print("\n---- Solución con variables de holgura visibles ----")
problema_holgura.print_solution(solucion_qubo_holgura, mostrar_holgura=True)

